In [1]:
#   ▄████████    ▄████████ ████████▄  ████████▄   ▄█      ███             ▄████████    ▄████████  ▄████████    ▄█    █▄     ▄█   ▄█    █▄     ▄████████    ▄████████
#   ███    ███   ███    ███ ███   ▀███ ███   ▀███ ███  ▀█████████▄        ███    ███   ███    ███ ███    ███   ███    ███   ███  ███    ███   ███    ███   ███    ███
#   ███    ███   ███    █▀  ███    ███ ███    ███ ███▌    ▀███▀▀██        ███    ███   ███    ███ ███    █▀    ███    ███   ███▌ ███    ███   ███    █▀    ███    ███
#  ▄███▄▄▄▄██▀  ▄███▄▄▄     ███    ███ ███    ███ ███▌     ███   ▀        ███    ███  ▄███▄▄▄▄██▀ ███         ▄███▄▄▄▄███▄▄ ███▌ ███    ███  ▄███▄▄▄      ▄███▄▄▄▄██▀
# ▀▀███▀▀▀▀▀   ▀▀███▀▀▀     ███    ███ ███    ███ ███▌     ███          ▀███████████ ▀▀███▀▀▀▀▀   ███        ▀▀███▀▀▀▀███▀  ███▌ ███    ███ ▀▀███▀▀▀     ▀▀███▀▀▀▀▀
# ▀███████████   ███    █▄  ███    ███ ███    ███ ███      ███            ███    ███ ▀███████████ ███    █▄    ███    ███   ███  ███    ███   ███    █▄  ▀███████████
#   ███    ███   ███    ███ ███   ▄███ ███   ▄███ ███      ███            ███    ███   ███    ███ ███    ███   ███    ███   ███  ███    ███   ███    ███   ███    ███
#   ███    ███   ██████████ ████████▀  ████████▀  █▀      ▄████▀          ███    █▀    ███    ███ ████████▀    ███    █▀    █▀    ▀██████▀    ██████████   ███    ███
#   ███    ███                                                                         ███    ███                                                          ███    ███

# Reddit Archiver - An easy way to archive and view your Reddit Account. 

Downloads your profile info and posts/comments from your input reddit account. Creates a presentable webpage of your arhcived profile that can be saved locally. 

* Allows user to input desired Reddit Username and output directory for archive file via command line
* Downloads user data using BeautifulSoup.
* Saves data to JSON file
* Loads JSON File into HTML webpage and saves webpage to desired directory


By [dandex200](https://github.com/dandex200/) on Github


#### Imports

In [72]:
import requests
from bs4 import BeautifulSoup
import json
import os
from tkinter import filedialog, Tk
import datetime
import time
import shutil
import wget
import youtube_dl
from moviepy.editor import VideoFileClip, AudioFileClip
import urllib.request

## User Input

In [73]:
#Username Input
user_input = input("Enter desired Reddit Username to Archive:")

#Output Directory Input using Tkinter
confirm_input = input("Choose Archive File Output Directory? (Y/N)")

#Select Output Directory if chosen
if confirm_input.lower() == 'y':
    root = Tk() 
    root.withdraw()
    output_dir = filedialog.askdirectory()
    root.destroy()
    print("Will save archive file to: %s" % output_dir)
else:
    print("No output directory selected - will save to current directory.")
    

Will save archive file to: /home/dandex/programming/The_Reddit_Archiver/Reddit_Archiver/Temp_Files


## Scraping

In [74]:
user_url = 'https://old.reddit.com/user/'+user_input

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

form_data = {"over18": "yes"}
response = requests.get(user_url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
title_check = soup.title.text.split(' ')[0]

if title_check == "overview":
    print("working")
else:
    print("Over 18, bypassing...")
    age_url =soup.select_one('link[rel="canonical"]')['href']
    print(age_url)
    new_response = requests.post(age_url, data=form_data, headers=headers)
    soup = BeautifulSoup(new_response.content, 'html.parser')
    title_check = soup.title.text.split(' ')[0]
    if title_check == "overview":
        print("bypass working")
    else:
        print("bypass failed")

working


### Profile Info

In [75]:
username = soup.title.text.split(' ')[2]

comment_karma_box = soup.find('span', {'class': 'comment-karma'})
comment_karma = comment_karma_box.text if comment_karma_box is not None else ''
comment_karma = int(comment_karma.replace(',', ''))
post_karma = soup.find('span', {'class': 'karma'})
post_karma = post_karma.text if post_karma is not None else ''
post_karma = int(post_karma.replace(',', ''))

user_creation_date = soup.find('span', {'class': 'age'})
user_creation_date = user_creation_date.find('time')['datetime']

num_posts = 0
num_comments = 0
for comment in soup.find_all('div', class_='comment'):
    num_comments += 1
for post in soup.find_all('div', class_='link'):
    num_posts += 1

print("Username: " + username)
print("Comment Karma: %d"  % comment_karma)
print("Post Karma: %d" % post_karma)
print("Creation Date: " + user_creation_date)
print("Posts Count: %d" % num_posts)
print("Comments Count: %d" % num_comments)

Username: Normal-Will4837
Comment Karma: 805
Post Karma: 123
Creation Date: 2022-06-09T05:50:13+00:00
Posts Count: 3
Comments Count: 22


### Comments

In [76]:
comments = []
comment_count = 0
spans = []
score = ""
score_hidden = ""
date_format = "%Y-%m-%dT%H:%M:%S%z"

for comment in soup.find_all('div', class_='comment'):
    utc_date = comment.find('time', class_="live-timestamp")['datetime']
    utc_date = datetime.datetime.strptime(utc_date, date_format)
    utc_date = utc_date.isoformat()
    spans = comment.find('span', class_='score unvoted')
    if spans is not None:
        score = spans.text
    else:
        score = comment.find('span', class_='score-hidden').text
    subreddit = comment.find('a', class_="subreddit").text
    permalink = comment.find('a', class_="bylink")['href']
    text = comment.find('div', class_='md-container').text
    comment_data = {
        'utc_date': utc_date,
        'score': score,
        'subreddit': subreddit,
        'permalink': permalink,
        'text': text
    }
    comments.append(comment_data)
    comment_count += 1

print(comments)
print(comment_count)

[{'utc_date': '2023-07-20T19:24:42+00:00', 'score': '1 point', 'subreddit': 'NoJumper', 'permalink': 'https://old.reddit.com/r/NoJumper/comments/153hfe1/is_that_a_voicecrack_i_heard_this_definitely/jsrpw5c/', 'text': 'the cream pie was fake and Adam playing into it to get a reaction/views and yall falling for it\n\n'}, {'utc_date': '2023-07-18T04:06:05+00:00', 'score': '4 points', 'subreddit': 'NoJumper', 'permalink': 'https://old.reddit.com/r/NoJumper/comments/152f0di/strawberry_deker_head_ass/jsevjkn/', 'text': 'She went to the same dr heather did\n\n'}, {'utc_date': '2023-07-17T04:32:16+00:00', 'score': '2 points', 'subreddit': 'NoJumper', 'permalink': 'https://old.reddit.com/r/NoJumper/comments/151d40x/naw_jason_luv_is_a_menace/jsa4wx6/', 'text': 'You don’t think Adam told him to play into the cuck narrative so this shit could go more viral and they all get a bigger bag off this shit? Lmao\n\n'}, {'utc_date': '2023-07-15T04:54:12+00:00', 'score': '1 point', 'subreddit': 'NoJumper',

### Posts

In [77]:
#Counting Posts
post_count = 0
for post in soup.find_all('div', class_='link'):
    print(post)
    post_count += 1
print(post_count)

<div class="thing id-t3_1558pmd over18 linkflair odd link" data-author="Normal-Will4837" data-author-fullname="t2_ocn76gdx" data-comments-count="7" data-context="listing" data-domain="i.redd.it" data-fullname="t3_1558pmd" data-gildings="0" data-is-gallery="false" data-nsfw="true" data-num-crossposts="0" data-oc="false" data-permalink="/r/NoJumper/comments/1558pmd/wait_until_blazzy_see_this_on_etsy/" data-promoted="false" data-rank="" data-score="42" data-spoiler="false" data-subreddit="NoJumper" data-subreddit-fullname="t5_3c1lr" data-subreddit-prefixed="r/NoJumper" data-subreddit-type="public" data-timestamp="1689900378000" data-type="link" data-url="https://i.redd.it/01eoww0gt7db1.jpg" data-whitelist-status="" id="thing_t3_1558pmd" onclick="click_thing(this)"><p class="parent"></p><span class="rank"></span><div class="midcol unvoted"><div aria-label="upvote" class="arrow up login-required access-required" data-event-action="upvote" role="button" tabindex="0"></div><div class="score d

In [78]:
output_dir

'/home/dandex/programming/The_Reddit_Archiver/Reddit_Archiver/Temp_Files'

In [79]:
#Identify Posts
posts = []
post_count = 0

utc_date = 0
post_type = ""
subreddit = ""
post_title = ""
file_path = ""
permalink = ""
score = 0
text = ""

#self post
for self_post in soup.find_all('div', class_='self'):
    post_count += 1
    utc_date = self_post.find('time', class_="live-timestamp")['datetime']
    utc_date = datetime.datetime.strptime(utc_date, date_format)
    utc_date = utc_date.isoformat()
    post_type = "self"
    subreddit = self_post.find('a', class_="subreddit").text
    post_title = self_post.find('a', class_="title").text
    score = self_post.find('div', {'class': 'score unvoted'}).text
    permalink = self_post.find('a', class_="bylink")['href']

    #Self text post (hardest part so far)
    if self_post.find('div', {'class': 'expando-button'}): #If expando button present
        #Download permalink
        #print("self expando")
        self_text_url = permalink
        print(self_text_url)

        headers2 = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
        response2 = requests.get(self_text_url, headers=headers2)
        soup2 = BeautifulSoup(response2.content, "html.parser")
        post_content = soup2.find("div", class_="entry")
        md_element = post_content.find("div", class_="md-container")
        post_text = md_element.text
        #expando = soup2.find("div", class_="usertext-body")
        #md_element = expando.find("div", class_="md-container")

        #md_element = expando.find("div", class_="md-container") if expando else None
        #md_text = md_element.text if md_element else None

        #post_text = md_text.text

        text = post_text
        print(post_text)
        
    else:
        text = None
    post_data = {
        'utc_date': utc_date,
        'score': score,
        'subreddit': subreddit,
        'permalink': permalink,
        'title': post_title,
        'type': post_type,
        'text': text
    }
    posts.append(post_data)
    
#crosspost
for cross_post in soup.find_all('div', {'class':'link', 'data-crosspost-root-title': True}):
    post_count += 1
    utc_date = cross_post.find('time', class_="live-timestamp")['datetime']
    utc_date = datetime.datetime.strptime(utc_date, date_format)
    utc_date = utc_date.isoformat()
    post_type = "crosspost"
    subreddit = cross_post.find('a', class_="subreddit").text
    post_title = cross_post.find('a', class_="title").text
    score = cross_post.find('div', {'class': 'score unvoted'}).text
    permalink = cross_post.find('a', class_="bylink")['href']
    cross_post_link = cross_post.find('a', class_="title")['href']
    post_data = {
        'utc_date': utc_date,
        'score': score,
        'subreddit': subreddit,
        'permalink': permalink,
        'title': post_title,
        'type': post_type,
        "CrossPost Link": cross_post_link
    }
    posts.append(post_data)
#link
domains = ['old.reddit.com', 'i.redd.it', 'v.redd.it', 'i.imgur.com']
for link_post in soup.find_all(lambda tag: tag.has_attr('data-domain') and not any(domain in tag['data-domain'] for domain in domains )and 'self' not in tag.get('class', [])):
    post_count += 1
    utc_date = link_post.find('time', class_="live-timestamp")['datetime']
    utc_date = datetime.datetime.strptime(utc_date, date_format)
    utc_date = utc_date.isoformat()
    post_type = "link"
    subreddit = link_post.find('a', class_="subreddit").text
    post_title = link_post.find('a', class_="title").text
    score = link_post.find('div', {'class': 'score unvoted'}).text
    permalink = link_post.find('a', class_="bylink")['href']
    
    #If expando, request permalink page, extract text, else text = NaN
    
    if link_post.find('div', {'class': 'expando-button'}): #If expando button present
        #Download permalink
        form_data = {"over18": "yes"}
        response = requests.get(permalink, data=form_data, headers={'User-Agent': 'Mozilla/5.0'})
        post_soup = BeautifulSoup(response.content, 'html.parser')
        #Extract text
        link = post_soup.find('a', class_="title")['href']
    else:
        link = None
    post_data = {
        'utc_date': utc_date,
        'score': score,
        'subreddit': subreddit,
        'permalink': permalink,
        'title': post_title,
        'type': post_type,
        'link': link
    }
    posts.append(post_data)
#video
for video_post in soup.find_all('div', {'class':'link', "data-domain": "v.redd.it", "data-kind": "video"}):
    post_count += 1
    utc_date = video_post.find('time', class_="live-timestamp")['datetime']
    utc_date = datetime.datetime.strptime(utc_date, date_format)
    utc_date = utc_date.isoformat()
    post_type = "video"
    subreddit = video_post.find('a', class_="subreddit").text
    post_title = video_post.find('a', class_="title").text
    score = video_post.find('div', {'class': 'score unvoted'}).text
    permalink = video_post.find('a', class_="bylink")['href']
    text = None
    vid_link = video_post.find('a', class_="title")['href']
    ydl_opts = {
        'outtmpl': f'{output_dir}/%(title)s.%(ext)s',
        #'outtmpl': output_dir + ,
        'merge_output_format': 'mp4',
    }
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        ydl.download([vid_link])
    #Merge and Remove 
    for filename in os.listdir(output_dir):
        if filename.endswith(".mp4") and "merged" not in filename:
            video_path = os.path.join(output_dir, filename)
            audio_filename_prefix = filename[:8]
            
            for file in os.listdir(output_dir):
                if file.startswith(audio_filename_prefix) and file.endswith(".m4a"):
                    audio_path = os.path.join(output_dir, file)
                    output_filename = os.path.splitext(filename)[0] + "_merged.mp4"
                    output_path = os.path.join(output_dir, output_filename)
                    video = VideoFileClip(video_path)
                    audio = AudioFileClip(audio_path)
                    video = video.set_audio(audio)
                    video.write_videofile(output_path, codec="libx264", audio_codec="aac")
                    break
            os.remove(video_path)
            os.remove(audio_path)
            vid_path = output_path
    post_data = {
        'utc_date': utc_date,
        'score': score,
        'subreddit': subreddit,
        'permalink': permalink,
        'title': post_title,
        'type': post_type,
        'filepath': vid_path
    }
    posts.append(post_data)
#picture 
for pic_post in soup.find_all('div', {'class':'link', "data-domain": "old.reddit.com"}):
    post_count += 1
    utc_date = pic_post.find('time', class_="live-timestamp")['datetime']
    utc_date = datetime.datetime.strptime(utc_date, date_format)
    utc_date = utc_date.isoformat()
    post_type = "pic"
    subreddit = pic_post.find('a', class_="subreddit").text
    post_title = pic_post.find('a', class_="title").text
    score = pic_post.find('div', {'class': 'score unvoted'}).text
    permalink = pic_post.find('a', class_="bylink")['href']
    pic_link = pic_post.find('a', class_="title")['href']
    pic_path = ""
    pic_filename = pic_link.split("/")[-1]  # Extract the original filename from the URL
    pic_path = os.path.join(output_dir, pic_filename)
    urllib.request.urlretrieve(pic_link, pic_path)
    post_data = {
        'utc_date': utc_date,
        'score': score,
        'subreddit': subreddit,
        'permalink': permalink,
        'title': post_title,
        'type': post_type,
        'filepath': pic_path
    }
    posts.append(post_data)

for pic_post in soup.find_all('div', {'class':'link', "data-domain": "i.redd.it"}):
    post_count += 1
    utc_date = pic_post.find('time', class_="live-timestamp")['datetime']
    utc_date = datetime.datetime.strptime(utc_date, date_format)
    utc_date = utc_date.isoformat()
    post_type = "pic"
    subreddit = pic_post.find('a', class_="subreddit").text
    post_title = pic_post.find('a', class_="title").text
    score = pic_post.find('div', {'class': 'score unvoted'}).text
    permalink = pic_post.find('a', class_="bylink")['href']
    pic_link = pic_post.find('a', class_="title")['href']
    pic_path = ""
    pic_filename = pic_link.split("/")[-1]  # Extract the original filename from the URL
    pic_path = os.path.join(output_dir, pic_filename)
    urllib.request.urlretrieve(pic_link, pic_path)
    post_data = {
        'utc_date': utc_date,
        'score': score,
        'subreddit': subreddit,
        'permalink': permalink,
        'title': post_title,
        'type': post_type,
        'filepath': pic_path
    }
    posts.append(post_data)

for pic_post in soup.find_all('div', {'class':'link', "data-domain": "i.imgur.com"}):
    post_count += 1
    utc_date = pic_post.find('time', class_="live-timestamp")['datetime']
    utc_date = datetime.datetime.strptime(utc_date, date_format)
    utc_date = utc_date.isoformat()
    post_type = "pic"
    subreddit = pic_post.find('a', class_="subreddit").text
    post_title = pic_post.find('a', class_="title").text
    score = pic_post.find('div', {'class': 'score unvoted'}).text
    permalink = pic_post.find('a', class_="bylink")['href']
    post_data = {
        'utc_date': utc_date,
        'score': score,
        'subreddit': subreddit,
        'permalink': permalink,
        'title': post_title,
        'type': post_type
    }
    posts.append(post_data)
print(post_count)

[Reddit] nydgldyaavab1: Downloading m3u8 information
[Reddit] nydgldyaavab1: Downloading MPD manifest


[download] Destination: /home/dandex/programming/The_Reddit_Archiver/Reddit_Archiver/Temp_Files/nydgldyaavab1.fdash-5.mp4
[download] 100% of 22.60MiB in 00:0405MiB/s ETA 00:004
[download] Destination: /home/dandex/programming/The_Reddit_Archiver/Reddit_Archiver/Temp_Files/nydgldyaavab1.fdash-6.m4a
[download] 100% of 1.98MiB in 00:0017MiB/s ETA 00:00known ETA
Moviepy - Building video /home/dandex/programming/The_Reddit_Archiver/Reddit_Archiver/Temp_Files/nydgldyaavab1.fdash-5_merged.mp4.
MoviePy - Writing audio in nydgldyaavab1.fdash-5_mergedTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /home/dandex/programming/The_Reddit_Archiver/Reddit_Archiver/Temp_Files/nydgldyaavab1.fdash-5_merged.mp4



Moviepy - Done !
Moviepy - video ready /home/dandex/programming/The_Reddit_Archiver/Reddit_Archiver/Temp_Files/nydgldyaavab1.fdash-5_merged.mp4
[Reddit] 7wocq0ca1uab1: Downloading m3u8 information
[Reddit] 7wocq0ca1uab1: Downloading MPD manifest


[download] Destination: /home/dandex/programming/The_Reddit_Archiver/Reddit_Archiver/Temp_Files/7wocq0ca1uab1.fdash-5.mp4
[download] 100% of 6.90MiB in 00:0015MiB/s ETA 00:00known ETA
[download] Destination: /home/dandex/programming/The_Reddit_Archiver/Reddit_Archiver/Temp_Files/7wocq0ca1uab1.fdash-6.m4a
[download] 100% of 505.71KiB in 00:0060MiB/s ETA 00:00known ETA
Moviepy - Building video /home/dandex/programming/The_Reddit_Archiver/Reddit_Archiver/Temp_Files/7wocq0ca1uab1.fdash-5_merged.mp4.
MoviePy - Writing audio in 7wocq0ca1uab1.fdash-5_mergedTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /home/dandex/programming/The_Reddit_Archiver/Reddit_Archiver/Temp_Files/7wocq0ca1uab1.fdash-5_merged.mp4



Moviepy - Done !
Moviepy - video ready /home/dandex/programming/The_Reddit_Archiver/Reddit_Archiver/Temp_Files/7wocq0ca1uab1.fdash-5_merged.mp4
3


In [41]:
#Merge mp4 and m4a files
for filename in os.listdir(output_dir):
    if filename.endswith(".mp4"):
        video_path = os.path.join(output_dir, filename)
        audio_filename_prefix = filename[:8]
        
        for file in os.listdir(output_dir):
            if file.startswith(audio_filename_prefix) and file.endswith(".m4a"):
                audio_path = os.path.join(output_dir, file)
                output_filename = os.path.splitext(filename)[0] + "_merged.mp4"
                output_path = os.path.join(output_dir, output_filename)
                video = VideoFileClip(video_path)
                audio = AudioFileClip(audio_path)
                video = video.set_audio(audio)
                video.write_videofile(output_path, codec="libx264", audio_codec="aac")
                break
        os.remove(video_path)
        os.remove(audio_path)

FileNotFoundError: [Errno 2] No such file or directory: '/home/dandex/programming/The_Reddit_Archiver/Reddit_Archiver/Temp_Files/414dcwng5lra1.fdash-audio_0_131828.m4a'

In [80]:
#Test if 25 Posts identified (Number of each page)
if (post_count + comment_count) != 25:
    print("Error")
else:
    
    print("working")

working


In [81]:
num_posts = len(posts)
num_comments = len(comments)

In [82]:
user_data = {
    'user_info': {
        'username': username,
        'user_creation_date': user_creation_date,
        'num_posts': num_posts,
        'num_comments': num_comments,
        'karma': {
            'post_karma': post_karma,
            'comment_karma': comment_karma
        }
    },
    'posts': posts,
    'comments': comments
}

In [83]:
with open('Temp_Files/user_data.json', 'w') as f:
    json.dump(user_data, f)

In [84]:
with open('Temp_Files/user_data.json') as f:
    data = json.load(f)

print(json.dumps(data, indent=4))


{
    "user_info": {
        "username": "Normal-Will4837",
        "user_creation_date": "2022-06-09T05:50:13+00:00",
        "num_posts": 3,
        "num_comments": 22,
        "karma": {
            "post_karma": 123,
            "comment_karma": 805
        }
    },
    "posts": [
        {
            "utc_date": "2023-07-09T04:30:29+00:00",
            "score": "19",
            "subreddit": "r/NoJumper",
            "permalink": "https://old.reddit.com/r/NoJumper/comments/14uoxw8/tokes_homie_sandman_and_vell_run_the_fade_after/",
            "title": "Toke\u2019s homie Sandman and Vell run the fade after Vell\u2019s girl butts her head in Sandman and his girls drama and gets called a nosey bitch (Allegedly)",
            "type": "video",
            "filepath": "/home/dandex/programming/The_Reddit_Archiver/Reddit_Archiver/Temp_Files/nydgldyaavab1.fdash-5_merged.mp4"
        },
        {
            "utc_date": "2023-07-09T00:17:22+00:00",
            "score": "58",
            "

**To-DO: PT1**
* Successfully append comments, user info, and posts of first page to JSON in correct format
* Save pictures/videos to directory
* Present JSON and saved files in beginner format HTML webpage
* Apply saving to all pages of reddit account
* Add tests to ensure data is all saved (meets 25 posts per page, etc)
* Create Readme on Github
* Publish first time

**TO-DO PT2**
* Convert from notebook to script
* Create GUI where you can choose if you want to save pics/videos, input user/output directory in GUI, etc
* Apply ability to save pics/videos as choice
* Create Pretty looking HTML archive page
* Promote on Reddit
* Release as second update 
* Add ability to keep link in comment

**TO-DO PT3**
* blank

**Minor Additions**
* Accounting for pinned posts
* Profile pictures (snoovatar??)
* Trophies
* Add over 18 bypass for old.reddit

In [14]:
#Clear Temp_Files
for filename in os.listdir(output_dir):
        file_path = os.path.join(output_dir, filename)
        if os.path.isfile(file_path):
            os.remove(file_path)